In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../restaurant-annual-turnover-model-exp/dataset/raw-data-restaurant.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3493 entries, 0 to 3492
Data columns (total 34 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Registration Number            3493 non-null   int64  
 1   Annual Turnover                3493 non-null   int64  
 2   Cuisine                        3493 non-null   object 
 3   City                           3493 non-null   object 
 4   Restaurant Location            3493 non-null   object 
 5   Opening Day of Restaurant      3493 non-null   object 
 6   Facebook Popularity Quotient   3394 non-null   float64
 7   Endorsed By                    3493 non-null   object 
 8   Instagram Popularity Quotient  3437 non-null   float64
 9   Fire Audit                     3493 non-null   int64  
 10  Liquor License Obtained        3493 non-null   int64  
 11  Situated in a Multi Complex    3493 non-null   int64  
 12  Dedicated Parking              3493 non-null   i

In [5]:
# Dropping columns
cols = ['Liquor License Obtained', 'Live Sports Rating', 'City']

df.drop(cols, axis=1, inplace=True)

In [6]:
df['Cuisine1'] = df['Cuisine'].apply(lambda x: x.split(',')[0])
df['Cuisine2'] = df['Cuisine'].apply(lambda x: x.split(',')[1])
df.drop('Cuisine', axis=1, inplace=True)

In [7]:
df['Opening Day of Restaurant'] = pd.to_datetime(df['Opening Day of Restaurant'], format='%d/%m/%y')

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3493 entries, 0 to 3492
Data columns (total 32 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   Registration Number            3493 non-null   int64         
 1   Annual Turnover                3493 non-null   int64         
 2   Restaurant Location            3493 non-null   object        
 3   Opening Day of Restaurant      3493 non-null   datetime64[ns]
 4   Facebook Popularity Quotient   3394 non-null   float64       
 5   Endorsed By                    3493 non-null   object        
 6   Instagram Popularity Quotient  3437 non-null   float64       
 7   Fire Audit                     3493 non-null   int64         
 8   Situated in a Multi Complex    3493 non-null   int64         
 9   Dedicated Parking              3493 non-null   int64         
 10  Open Sitting Available         3493 non-null   int64         
 11  Resturant Tier   

In [9]:
df.head()

,Registration Number,Annual Turnover,Restaurant Location,Opening Day of Restaurant,Facebook Popularity Quotient,Endorsed By,Instagram Popularity Quotient,Fire Audit,Situated in a Multi Complex,Dedicated Parking,...,Live Music Rating,Comedy Gigs Rating,Value Deals Rating,Ambience,Lively,Service,Comfortablility,Privacy,Cuisine1,Cuisine2
0,60001,42000000,Near Business Hub,2009-02-14,84.3,Not Specific,95.8,1,1,0,...,4.0,NaN,NaN,8.0,8,6,6,6,indian,irish
1,60002,50000000,Near Party Hub,2008-09-29,85.4,Tier A Celebrity,85.0,1,1,1,...,NaN,4.0,NaN,5.0,7,7,3,8,indian,irish
2,60003,32500000,Near Business Hub,2011-07-30,85.0,Tier A Celebrity,68.2,1,1,1,...,3.0,NaN,NaN,7.0,10,5,2,8,tibetan,italian
3,60004,110000000,Near Party Hub,2008-11-30,85.6,Tier A Celebrity,83.6,0,1,1,...,6.0,NaN,NaN,7.0,7,4,3,5,turkish,nigerian
4,60005,20000000,Near Party Hub,2010-02-22,NaN,Tier A Celebrity,76.8,1,1,1,...,NaN,2.0,NaN,NaN,6,2,4,6,irish,belgian


In [20]:
import hopsworks
project = hopsworks.login()
fs = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/514196
Connected. Call `.close()` to terminate connection gracefully.


In [10]:
# rename every column in snake case
df.columns = df.columns.str.replace(' ', '_').str.lower()

In [11]:
df.columns

Index(['registration_number', 'annual_turnover', 'restaurant_location',
       'opening_day_of_restaurant', 'facebook_popularity_quotient',
       'endorsed_by', 'instagram_popularity_quotient', 'fire_audit',
       'situated_in_a_multi_complex', 'dedicated_parking',
       'open_sitting_available', 'resturant_tier', 'restaurant_type',
       'restaurant_theme', 'restaurant_zomato_rating', 'restaurant_city_tier',
       'order_wait_time', 'staff_responsivness', 'value_for_money',
       'hygiene_rating', 'food_rating', 'overall_restaurant_rating',
       'live_music_rating', 'comedy_gigs_rating', 'value_deals_rating',
       'ambience', 'lively', 'service', 'comfortablility', 'privacy',
       'cuisine1', 'cuisine2'],
      dtype='object')

In [75]:
# Get or create the 'iris' feature group
restaurant_fg = fs.get_or_create_feature_group(
    name="restaurant_fg",
    version=1,
    online_enabled=True,
    primary_key=["registration_number"],
    description="Restaurant feature group",
)
# Insert data info the feature group
restaurant_fg.insert(df)

Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/514196/fs/510019/fg/606602


Uploading Dataframe: 0.00% |          | Rows 0/3493 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: restaurant_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/514196/jobs/named/restaurant_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7f499c33c090>, None)

In [76]:
label_encoding_cols = ['restaurant_location', 'endorsed_by', 'restaurant_type', 'restaurant_theme', 'cuisine1', 'cuisine2']
label_encoder = fs.get_transformation_function(name="label_encoder")
transform_function_dict = {col: label_encoder for col in label_encoding_cols}

In [77]:
restaurant_fg = fs.get_feature_group('restaurant_fg', 1)

In [ ]:
# Feature View Creation
selected_features = restaurant_fg.select_all()
# Get or create the 'iris' feature view
restaurant_fv = fs.get_or_create_feature_view(
    name="restaurant_fv",
    version=1,
    labels=['annual_turnover'],
    description="Restaurant feature view",
    query=selected_features,
    transformation_functions=transform_function_dict,
)

In [30]:
restaurant_fv = fs.get_feature_view('restaurant_fv', 1)

In [79]:
version, job = restaurant_fv.create_training_data(
    description='Description of a dataset',
    data_format='csv',
    # async creation in order not to wait till finish of the job
    write_options={"wait_for_job": True}
)

Training dataset job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/514196/jobs/named/restaurant_fv_1_create_fv_td_12032024043408/executions


In [31]:
features_df, labels_df = restaurant_fv.get_training_data(training_dataset_version=1)

In [35]:
features_df.head()

,registration_number,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,situated_in_a_multi_complex,dedicated_parking,open_sitting_available,...,live_music_rating,comedy_gigs_rating,value_deals_rating,ambience,lively,service,comfortablility,privacy,cuisine1,cuisine2
0,62821,1,2010-12-08T00:00:00.000Z,80.80,0,89.00,1,1,1,1,...,4.0,NaN,NaN,6.0,6,5,4,2,9,9
1,60832,1,2010-11-12T00:00:00.000Z,83.30,2,81.00,0,1,1,1,...,4.0,NaN,NaN,6.0,7,5,2,5,3,11
2,62290,0,2009-03-13T00:00:00.000Z,89.60,0,80.33,1,1,1,0,...,6.0,NaN,NaN,9.0,9,3,0,7,13,0
3,62768,1,2010-02-23T00:00:00.000Z,62.50,0,60.00,1,0,0,0,...,3.0,2.0,NaN,6.0,5,4,2,4,11,4
4,62362,0,2007-03-26T00:00:00.000Z,82.26,0,81.33,1,0,0,1,...,NaN,4.0,NaN,9.0,9,7,7,9,10,6


In [85]:
labels_df.head()

,annual_turnover
0,40000000
1,32000000
2,49000000
3,16500000
4,31000000


In [34]:
restaurant_fv.get_feature_vector({"registration_number": 60001})

[60001,
 0,
 datetime.datetime(2009, 2, 14, 0, 0),
 84.3,
 0,
 95.8,
 1,
 1,
 0,
 1,
 2.0,
 0,
 26,
 3,
 0,
 2,
 5,
 5,
 5,
 7,
 10.0,
 4.0,
 None,
 None,
 8.0,
 8,
 6,
 6,
 13,
 0,
 12]

In [24]:
# modify 10 values of datframe of col privacy
df.loc[0:10, 'privacy'] = 13

In [25]:
df.head()

,registration_number,annual_turnover,restaurant_location,opening_day_of_restaurant,facebook_popularity_quotient,endorsed_by,instagram_popularity_quotient,fire_audit,situated_in_a_multi_complex,dedicated_parking,...,live_music_rating,comedy_gigs_rating,value_deals_rating,ambience,lively,service,comfortablility,privacy,cuisine1,cuisine2
0,60001,42000000,Near Business Hub,2009-02-14,84.3,Not Specific,95.8,1,1,0,...,4.0,NaN,NaN,8.0,8,6,6,13,indian,irish
1,60002,50000000,Near Party Hub,2008-09-29,85.4,Tier A Celebrity,85.0,1,1,1,...,NaN,4.0,NaN,5.0,7,7,3,13,indian,irish
2,60003,32500000,Near Business Hub,2011-07-30,85.0,Tier A Celebrity,68.2,1,1,1,...,3.0,NaN,NaN,7.0,10,5,2,13,tibetan,italian
3,60004,110000000,Near Party Hub,2008-11-30,85.6,Tier A Celebrity,83.6,0,1,1,...,6.0,NaN,NaN,7.0,7,4,3,13,turkish,nigerian
4,60005,20000000,Near Party Hub,2010-02-22,NaN,Tier A Celebrity,76.8,1,1,1,...,NaN,2.0,NaN,NaN,6,2,4,13,irish,belgian


In [26]:
restaurant_fg = fs.get_feature_group('restaurant_fg', 1)

In [27]:
restaurant_fg.insert(df.loc[0:10])

Uploading Dataframe: 0.00% |          | Rows 0/11 | Elapsed Time: 00:00 | Remaining Time: ?

Launching job: restaurant_fg_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/514196/jobs/named/restaurant_fg_1_offline_fg_materialization/executions


(<hsfs.core.job.Job at 0x7faefdf42dd0>, None)